In [ ]:
import requests              # Importa la librería para hacer solicitudes HTTP (GET, POST, etc.) a APIs
import pandas as pd          # Importa la librería pandas, útil para manejar y analizar datos en forma de tablas (DataFrames)
import numpy as np           # Importa numpy, librería para cálculos numéricos y manejo de arreglos
from keys import *           # Importa las claves de API (por ejemplo OWM_key) almacenadas en el archivo "keys.py"

city = "Berlin"              # Define la ciudad para la cual se consultará el pronóstico del clima (en este caso, Berlín)
country = "DE"               # Define el código del país en formato ISO (DE = Alemania)

# Realiza una solicitud GET a la API de OpenWeatherMap usando la URL del endpoint "forecast"
# El parámetro "q" indica la ciudad y el país
# "appid" incluye la clave de API (OWM_key) necesaria para autenticarse
# "units=metric" indica que la temperatura se obtendrá en grados Celsius
# "lang=en" especifica que las descripciones del clima estarán en inglés
response = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city},{country}&appid={OWM_key}&units=metric&lang=en")

In [ ]:
data = response.json()  
# Convierte la respuesta de la API (en formato JSON) en un diccionario de Python.
# Ahora 'data' contiene toda la información del pronóstico del clima.

# Extrae la lista de pronósticos desde la clave 'list' del JSON.
# Cada elemento dentro de 'list' representa una entrada del clima (cada 3 horas aproximadamente).
forecast_list = data.get('list', [])

# =============================
# Preparación de listas vacías
# =============================
# Estas listas almacenarán los valores que luego se organizarán en columnas de un DataFrame.
times = []               # Fechas y horas de cada pronóstico
temperatures = []        # Temperaturas en grados Celsius
humidities = []          # Porcentaje de humedad
weather_statuses = []    # Estado del clima (por ejemplo: Clear, Clouds, Rain)
wind_speeds = []         # Velocidad del viento en m/s
rain_volumes = []        # Volumen de lluvia acumulado en 3 horas (mm)
snow_volumes = []        # Volumen de nieve acumulado en 3 horas (mm)

# =============================
# Extracción de datos del JSON
# =============================
# Se recorre cada entrada dentro de la lista de pronósticos.
for entry in forecast_list:
    # Obtiene la fecha y hora del pronóstico (clave 'dt_txt').
    times.append(entry.get('dt_txt', np.nan))
    
    # Obtiene la temperatura dentro del bloque 'main'.
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    
    # Obtiene la humedad (en porcentaje) dentro del bloque 'main'.
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    
    # Obtiene el estado principal del clima dentro del bloque 'weather' (ejemplo: Rain, Clear, Clouds).
    # Se usa [0] porque 'weather' es una lista con un solo elemento.
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    
    # Obtiene la velocidad del viento dentro del bloque 'wind'.
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    
    # Obtiene el volumen de lluvia en las últimas 3 horas (si existe).
    rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
    
    # Obtiene el volumen de nieve en las últimas 3 horas (si existe).
    snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

# =============================
# Creación del DataFrame
# =============================
# Se construye un DataFrame de pandas con las listas previamente llenadas.
# Cada lista se convierte en una columna con su respectivo nombre.
df = pd.DataFrame({
    'time': times,                   # Fecha y hora del pronóstico
    'temperature': temperatures,     # Temperatura (°C)
    'humidity': humidities,          # Humedad relativa (%)
    'weather_status': weather_statuses,  # Estado general del clima
    'wind_speed': wind_speeds,       # Velocidad del viento (m/s)
    'rain_volume_3h': rain_volumes,  # Lluvia en los últimos 3h (mm)
    'snow_volume_3h': snow_volumes   # Nieve en los últimos 3h (mm)
})

# =============================
# Visualización de resultados
# =============================
# Muestra las primeras cinco filas del DataFrame para verificar los datos.
print(df.head())


                  time  temperature  humidity weather_status  wind_speed  \
0  2025-10-05 00:00:00         8.56        82           Rain        8.29   
1  2025-10-05 03:00:00         8.65        81         Clouds        7.66   
2  2025-10-05 06:00:00         9.37        74         Clouds        6.89   
3  2025-10-05 09:00:00        11.65        66         Clouds        6.34   
4  2025-10-05 12:00:00        15.02        52           Rain        7.31   

   rain_volume_3h  snow_volume_3h  
0            0.64             NaN  
1             NaN             NaN  
2             NaN             NaN  
3             NaN             NaN  
4            0.31             NaN  
